# Model Improvement

## feature engineering

In [52]:
import pandas as pd
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestRegressor

In [53]:
df = pd.read_csv('../data/train.csv')

In [59]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    res = df.copy()
    res["clean_title"] = res["clean_title"].fillna("NaN")
    res["is.clean_title"] = res["clean_title"] == "Yes"
    res["age"] = (2024 - res["model_year"]).map(lambda x: max(x, 1))
    res["milage_per_year"] = res["milage"] / res["age"]
    res["had_accident"] = res["accident"] == "At least 1 accident or damage reported"
    res["avg_price"] =  res.groupby(['brand', 'model'])['price'].transform('mean')
    res["model_year"] = res["model_year"]
    return res

In [60]:
from sklearn.model_selection import train_test_split
preprocessed = preprocess(df)
train, val = train_test_split(preprocessed, test_size = 0.3, random_state = 42)

In [41]:
X_train = train[["is.clean_title", "milage_per_year", "had_accident", "avg_price", "model_year"]]
y_train = train["price"]

In [9]:
from sklearn.ensemble import RandomForestRegressor

In [28]:
forest = RandomForestRegressor(n_estimators = 100, random_state = 42)
forest.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [29]:
X_val = val[["is.clean_title", "milage_per_year", "had_accident", "avg_price", "model_year"]]
y_val = val["price"]

In [30]:
y_pred = forest.predict(X_val)

In [31]:
result = pd.DataFrame({"y_val": y_val, "y_pred": y_pred})
result["diff"] = result["y_val"] - result["y_pred"]

In [32]:
result

,y_val,y_pred,diff
111355,37999,35211.280000,2787.720000
182258,30000,32321.300667,-2321.300667
14147,26772,44324.573452,-17552.573452
79313,24999,20533.440000,4465.560000
101160,47995,32936.563333,15058.436667
...,...,...,...
95824,26500,38369.716667,-11869.716667
183599,54500,47203.700833,7296.299167
10343,10000,12931.540000,-2931.540000
76346,22989,27017.230000,-4028.230000


In [33]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [34]:
np.sqrt(mean_squared_error(y_val, y_pred))

np.float64(72442.71811240997)

# Submisstion

In [35]:
test = pd.read_csv('../data/test.csv')
test_preprocessed = preprocess(test)

KeyError: 'Column not found: price'

In [ ]:
X_test = test_preprocessed[["is.clean_title", "milage_per_year", "had_accident"]]

In [ ]:
y_submit = forest.predict(X_test)

In [ ]:
test["price"] = y_submit

In [ ]:
test[["id", "price"]].to_csv("../submission/20240910-random-forest.csv", index = False)